In [57]:
import fastf1
import plotly.express as px
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt

from datetime import timedelta


In [20]:
session = fastf1.get_session(2021, 'Monaco', 'R')
session

2021 Season Round 5: Monaco Grand Prix - Race

In [21]:
session.load()

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.0.4]
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written 

In [135]:
big_laps = pd.read_csv("../raw_data/laps.csv")
weather = pd.read_csv("../raw_data/weather.csv")
track = pd.read_csv("../raw_data/track_status.csv")

#fastf1.get_event_schedule(2017)
#big_laps['Compound'].fillna(method="bfill",limit=3,inplace=True)

#big_laps[['Compound','Year',"LapNumber"]][big_laps['Compound'].isna() & big_laps['LapNumber']==1]


/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_74931/3999582269.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  big_laps = pd.read_csv("../raw_data/laps.csv")


In [136]:
def resample_weather(weather_df,backfilling=3):
    print(len(weather_df))
    weather_df = weather_df.drop_duplicates(subset=['Location','Year','Time_min'], keep='last')
    print(len(weather_df))
    print(len(weather_df))
    years = weather_df['Year'].unique()
    output = pd.DataFrame()
    for year in years:
            year_df = weather_df.loc[weather_df.Year == year]
            locations = year_df['Location'].unique()
            # Loop over every location
            for location in locations:
                loc_df = year_df.loc[(year_df.Location == location)]
                loc_df.fillna(method="bfill",limit=backfilling,inplace=True)
                output = pd.concat([output,loc_df])
    return output

In [142]:
def merge_weather(laps, weather,backfilling=6):
    def add_standardized_time(df):
        df['Time_min']=pd.to_timedelta(df['Time'])
        df['Time_min']=df['Time_min'].values.astype('timedelta64[m]')
    add_standardized_time(laps)
    add_standardized_time(weather)
    laps_extended = laps.merge(weather,on=["Year","Location","Time_min"],how="left",suffixes=(None,"_w"))
    for column in weather.columns:
        laps_extended[column].fillna(method="bfill",limit=backfilling,inplace=True)
    return laps_extended

laps_extended=merge_weather(big_laps,weather)
laps_extended

laps_extended[laps_extended['AirTemp'].isna()]['LapNumber'].unique()

array([56., 65.,  9., 10., 11., 12., 13., 14., 57., 69.])

In [133]:
laps_extended[(laps_extended.Location=="Shanghai") & (laps_extended.Year==2019) & (laps_extended.Driver=="GAS")]

,Unnamed: 0,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,...,Time_min,Unnamed: 0_w,Time_w,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
27145,24741,0 days 00:35:21.497000,GAS,10,0 days 00:01:54.122000,1.0,1.0,0 days 00:00:04.213000,NaN,NaN,...,0 days 00:35:00,2640.0,0 days 00:35:36.746000,19.4,45.1,1019.1,False,28.9,79.0,2.6
27146,24741,0 days 00:35:21.497000,GAS,10,0 days 00:01:54.122000,1.0,1.0,0 days 00:00:04.213000,NaN,NaN,...,0 days 00:35:00,5346.0,0 days 00:35:36.746000,19.4,45.1,1019.1,False,28.9,79.0,2.6
27147,24742,0 days 00:37:08.906000,GAS,10,0 days 00:01:47.409000,2.0,1.0,NaN,NaN,0 days 00:00:32.871000,...,0 days 00:37:00,2642.0,0 days 00:37:37.047000,19.8,43.4,1019.1,False,28.9,84.0,3.2
27148,24742,0 days 00:37:08.906000,GAS,10,0 days 00:01:47.409000,2.0,1.0,NaN,NaN,0 days 00:00:32.871000,...,0 days 00:37:00,5348.0,0 days 00:37:37.047000,19.8,43.4,1019.1,False,28.9,84.0,3.2
27149,24743,0 days 00:38:48.727000,GAS,10,0 days 00:01:39.821000,3.0,1.0,NaN,NaN,0 days 00:00:26.332000,...,0 days 00:38:00,2643.0,0 days 00:38:36.597000,19.7,43.3,1019.1,False,28.5,92.0,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27251,24794,0 days 02:03:50.616000,GAS,10,0 days 00:01:56.679000,54.0,4.0,0 days 02:02:14.594000,NaN,0 days 00:00:44.577000,...,0 days 02:03:00,2728.0,0 days 02:03:38.090000,19.0,50.4,1019.3,False,25.3,106.0,3.3
27252,24794,0 days 02:03:50.616000,GAS,10,0 days 00:01:56.679000,54.0,4.0,0 days 02:02:14.594000,NaN,0 days 00:00:44.577000,...,0 days 02:03:00,5434.0,0 days 02:03:38.090000,19.0,50.4,1019.3,False,25.3,106.0,3.3
27253,24795,0 days 02:05:25.358000,GAS,10,0 days 00:01:34.742000,55.0,4.0,NaN,NaN,0 days 00:00:25.012000,...,0 days 02:05:00,2730.0,0 days 02:05:37.817000,19.0,51.1,1019.3,False,25.2,76.0,3.2
27254,24795,0 days 02:05:25.358000,GAS,10,0 days 00:01:34.742000,55.0,4.0,NaN,NaN,0 days 00:00:25.012000,...,0 days 02:05:00,5436.0,0 days 02:05:37.817000,19.0,51.1,1019.3,False,25.2,76.0,3.2


In [80]:
track['Status'].unique()

array([2, 1, 6, 4, 7, 5])

In [51]:
laps_extended['LapStartTime']

0           0 days 00:07:07.988000
1           0 days 00:07:07.988000
2           0 days 00:07:07.988000
3           0 days 00:07:07.988000
4           0 days 00:07:07.988000
                     ...          
14781318    0 days 02:29:45.580000
14781319    0 days 02:29:45.580000
14781320    0 days 02:29:45.580000
14781321    0 days 02:29:45.580000
14781322    0 days 02:29:45.580000
Name: LapStartTime, Length: 14781323, dtype: object

In [16]:
column_dict = { column: index+1 for index, column in enumerate(big_laps.columns)}
column_dict

{'Unnamed: 0': 1,
 'Time': 2,
 'Driver': 3,
 'DriverNumber': 4,
 'LapTime': 5,
 'LapNumber': 6,
 'Stint': 7,
 'PitOutTime': 8,
 'PitInTime': 9,
 'Sector1Time': 10,
 'Sector2Time': 11,
 'Sector3Time': 12,
 'Sector1SessionTime': 13,
 'Sector2SessionTime': 14,
 'Sector3SessionTime': 15,
 'SpeedI1': 16,
 'SpeedI2': 17,
 'SpeedFL': 18,
 'SpeedST': 19,
 'IsPersonalBest': 20,
 'Compound': 21,
 'TyreLife': 22,
 'FreshTyre': 23,
 'Team': 24,
 'LapStartTime': 25,
 'LapStartDate': 26,
 'TrackStatus': 27,
 'Position': 28,
 'Deleted': 29,
 'DeletedReason': 30,
 'FastF1Generated': 31,
 'IsAccurate': 32,
 'Location': 33,
 'Year': 34,
 'close_ahead': 35,
 'close_behind': 36}

In [24]:
big_laps['pitting_this_lap'] = np.where(big_laps['PitInTime'].notna(), True, False)
big_laps

,Unnamed: 0,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,...,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Location,Year,pitting_this_lap
0,0,0 days 00:08:53.241000,GAS,10,0 days 00:01:45.060000,1.0,NaN,0 days 00:00:04.103000,NaN,NaN,...,NaN,1.0,17.0,False,NaN,False,False,Melbourne,2018,False
1,1,0 days 00:10:26.613000,GAS,10,0 days 00:01:33.372000,2.0,1.0,NaN,NaN,0 days 00:00:31.357000,...,NaN,1.0,17.0,False,NaN,False,True,Melbourne,2018,False
2,2,0 days 00:11:59.474000,GAS,10,0 days 00:01:32.861000,3.0,1.0,NaN,NaN,0 days 00:00:31.160000,...,NaN,1.0,17.0,False,NaN,False,True,Melbourne,2018,False
3,3,0 days 00:13:31.658000,GAS,10,0 days 00:01:32.184000,4.0,1.0,NaN,NaN,0 days 00:00:30.835000,...,NaN,1.0,17.0,False,NaN,False,True,Melbourne,2018,False
4,4,0 days 00:15:03.990000,GAS,10,0 days 00:01:32.332000,5.0,1.0,NaN,NaN,0 days 00:00:30.716000,...,NaN,2.0,17.0,False,NaN,False,True,Melbourne,2018,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125754,125754,0 days 02:25:43.046000,SAR,2,0 days 00:01:21.280000,61.0,3.0,NaN,NaN,0 days 00:00:24.354000,...,2023-06-04 14:25:21.773,1.0,20.0,False,NaN,False,True,Barcelona,2023,False
125755,125755,0 days 02:27:05.180000,SAR,2,0 days 00:01:22.134000,62.0,3.0,NaN,NaN,0 days 00:00:23.675000,...,2023-06-04 14:26:43.053,1.0,20.0,False,NaN,False,True,Barcelona,2023,False
125756,125756,0 days 02:28:25.600000,SAR,2,0 days 00:01:20.420000,63.0,3.0,NaN,NaN,0 days 00:00:23.634000,...,2023-06-04 14:28:05.187,1.0,20.0,False,NaN,False,True,Barcelona,2023,False
125757,125757,0 days 02:29:45.580000,SAR,2,0 days 00:01:19.980000,64.0,3.0,NaN,NaN,0 days 00:00:23.602000,...,2023-06-04 14:29:25.607,1.0,20.0,False,NaN,False,True,Barcelona,2023,False


In [25]:

close_timedelta = timedelta(seconds=1,milliseconds=500)
big_laps['close_ahead']=False
big_laps['close_behind']=False
big_laps['is_pitting_ahead']=False
big_laps['is_pitting_behind']=False

column_dict = { column: index+1 for index, column in enumerate(big_laps.columns)}

for lap in big_laps[:1000].itertuples():
    position = lap[column_dict['Position']]
    track = lap[column_dict['Location']]
    year = lap[column_dict['Year']]
    lapnum = lap[column_dict['LapNumber']]
    curr_time = pd.to_timedelta(lap[column_dict['Time']])
    position_ahead = position-1
    position_behind = position+1
    if position_ahead >= 1:
        ahead = big_laps.loc[(big_laps['Position']==position_ahead) & (big_laps['Location']==track) & (big_laps['Year']==year) & (big_laps['LapNumber']==lapnum)]
        print(ahead)
        ahead_pit = ahead.iloc[0]['pitting_this_lap']
        ahead_time = pd.to_timedelta(ahead.iloc[0]['Time'])
        delta_ahead = curr_time-ahead_time
        if delta_ahead>=close_timedelta:
            big_laps.loc[lap[0],"close_ahead"]=True
        big_laps.loc[lap[0],"is_pitting_ahead"]=ahead_pit
    if position_behind <=20:
        behind = big_laps.loc[(big_laps['Position']==position_behind) & (big_laps['Location']==track) & (big_laps['Year']==year) & (big_laps['LapNumber']==lapnum)]
        if len(behind)>0:
            behind_pit = behind.iloc[0]['pitting_this_lap']
            behind_time = pd.to_timedelta(behind.iloc[0]['Time'])
            delta_behind = behind_time-curr_time
            if delta_behind>=close_timedelta:
                big_laps.loc[lap[0],"close_behind"]=True
            big_laps.loc[lap[0],"is_pitting_behind"]=behind_pit
        

     Unnamed: 0                    Time Driver  DriverNumber  \
934         934  0 days 00:08:52.855000    ERI             9   

                    LapTime  LapNumber  Stint              PitOutTime  \
934  0 days 00:01:44.674000        1.0    NaN  0 days 00:00:04.103000   

    PitInTime Sector1Time  ... DeletedReason FastF1Generated IsAccurate  \
934       NaN         NaN  ...           NaN           False      False   

      Location  Year  pitting_this_lap  close_ahead  close_behind  \
934  Melbourne  2018             False        False         False   

     is_pitting_ahead is_pitting_behind  
934             False             False  

[1 rows x 39 columns]
     Unnamed: 0                    Time Driver  DriverNumber  \
935         935  0 days 00:10:26.032000    ERI             9   

                    LapTime  LapNumber  Stint PitOutTime PitInTime  \
935  0 days 00:01:33.177000        2.0    1.0        NaN       NaN   

                Sector1Time  ... DeletedReason FastF1Gene

In [27]:
big_laps[big_laps['is_pitting_ahead']==True]

,Unnamed: 0,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,...,DeletedReason,FastF1Generated,IsAccurate,Location,Year,pitting_this_lap,close_ahead,close_behind,is_pitting_ahead,is_pitting_behind
35,35,0 days 00:40:40.960000,PER,11,0 days 00:01:31.254000,22.0,1.0,NaN,NaN,0 days 00:00:30.525000,...,NaN,False,True,Melbourne,2018,False,False,False,True,False
97,97,0 days 00:47:19.145000,ALO,14,0 days 00:02:20.530000,26.0,1.0,NaN,0 days 00:46:59.732000,0 days 00:00:42.712000,...,NaN,False,False,Melbourne,2018,True,True,True,True,False
151,151,0 days 00:41:20.722000,LEC,16,0 days 00:01:33.591000,22.0,2.0,NaN,NaN,0 days 00:00:29.843000,...,NaN,False,True,Melbourne,2018,False,True,True,True,True
152,152,0 days 00:42:50.690000,LEC,16,0 days 00:01:29.968000,23.0,2.0,NaN,NaN,0 days 00:00:30.302000,...,NaN,False,True,Melbourne,2018,False,True,True,True,False
154,154,0 days 00:46:11.636000,LEC,16,0 days 00:01:51.623000,25.0,2.0,NaN,NaN,0 days 00:00:30.268000,...,NaN,False,False,Melbourne,2018,False,True,True,True,False
158,158,0 days 00:54:19.785000,LEC,16,0 days 00:01:35.511000,29.0,3.0,NaN,NaN,0 days 00:00:33.601000,...,NaN,False,False,Melbourne,2018,False,True,True,True,False
211,211,0 days 00:43:57.381000,STR,18,0 days 00:01:30.955000,24.0,1.0,NaN,NaN,0 days 00:00:30.565000,...,NaN,False,True,Melbourne,2018,False,True,True,True,False
321,321,0 days 00:34:22.513000,MAG,20,0 days 00:01:30.042000,18.0,1.0,NaN,NaN,0 days 00:00:30.085000,...,NaN,False,True,Melbourne,2018,False,True,False,True,False
350,350,0 days 00:45:28.275000,HUL,27,0 days 00:01:43.870000,25.0,2.0,0 days 00:43:47.387000,NaN,0 days 00:00:35.200000,...,NaN,False,False,Melbourne,2018,False,True,False,True,True
389,389,0 days 00:17:00.886000,HAR,28,0 days 00:01:32.111000,6.0,2.0,NaN,NaN,0 days 00:00:30.461000,...,NaN,False,True,Melbourne,2018,False,True,False,True,False


In [66]:
start_2023 = big_laps[(big_laps['LapNumber']==1) & (big_laps['Year']==2018)]
start_before = big_laps[(big_laps['LapNumber']==2) & (big_laps['Year']<2023)]
big_laps.columns
#plt.hist(start_2023['Compound'])
#plt.hist(start_before['Compound'])

Index(['Unnamed: 0', 'Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber',
       'Stint', 'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time',
       'Sector3Time', 'Sector1SessionTime', 'Sector2SessionTime',
       'Sector3SessionTime', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST',
       'IsPersonalBest', 'Compound', 'TyreLife', 'FreshTyre', 'Team',
       'LapStartTime', 'LapStartDate', 'TrackStatus', 'Position', 'Deleted',
       'DeletedReason', 'FastF1Generated', 'IsAccurate', 'Location', 'Year'],
      dtype='object')

In [171]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


big_laps["second_compound"]=False
big_laps["second_compound"][big_laps['Stint']==1] = False

norris = big_laps[(big_laps['Driver']=="NOR")]


norris = norris[['Driver','LapNumber','Compound','Location','Year']][(norris.Location=="Barcelona") & (norris.Year==2023)]
#norris["second_compound"] = norris.apply(lambda x: 
#len(norris['Compound'].unique())

norris["prev_compound"]=norris["Compound"].shift(1)

big_laps = big_laps[(big_laps.Location=="Barcelona") & (big_laps.Year==2023)]
years = big_laps['Year'].unique()
for year in years:
    year_df = big_laps.loc[big_laps.Year == year]
    locations = year_df['Location'].unique()
    for location in locations:
        loc_df = year_df.loc[(year_df.Location == location)]
        drivers = loc_df['DriverNumber'].unique()
        for driver in drivers:
            driver_df = loc_df.loc[(loc_df.DriverNumber == driver)]
            driver_df["prev_compound"]=driver_df["Compound"].shift(1)
            #driver_df["prev_second_compound"]=driver_df["second_compound"].shift(1)
            #print(driver_df[['Driver','LapNumber','Compound','Location','Year','prev_compound']]
            first_index = driver_df.index[0]
            for index, row in driver_df.iterrows():
                if index-1>=first_index:
                    if big_laps.loc[index-1,"second_compound"]==True :
                        print("Hello1")
                        print(index)
                        print(big_laps.loc[index-1,"second_compound"])
                        big_laps.loc[index,"second_compound"]=True
                        print(big_laps.loc[index,"second_compound"])
                if big_laps.loc[index,"second_compound"]!=True:
                    if row["Compound"] != row["prev_compound"] and row["prev_compound"]:
                        print("Hello2")
                        print(row)
                        big_laps.loc[index,"second_compound"]=True
                        print(driver_df.loc[index,"second_compound"])

big_laps[big_laps.Driver=="VER"]

/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/3175505343.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_laps["second_compound"][big_laps['Stint']==1] = False
/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/3175505343.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_df["prev_compound"]=driver_df["Compound"].shift(1)
/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/3175505343.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

Hello2
Unnamed: 0                             124473
Time                   0 days 01:38:47.957000
Driver                                    VER
DriverNumber                                1
LapTime                0 days 00:01:38.427000
LapNumber                                27.0
Stint                                     2.0
PitOutTime             0 days 01:37:29.019000
PitInTime                                 NaN
Sector1Time            0 days 00:00:43.370000
Sector2Time            0 days 00:00:31.614000
Sector3Time            0 days 00:00:23.443000
Sector1SessionTime     0 days 01:37:52.900000
Sector2SessionTime     0 days 01:38:24.514000
Sector3SessionTime     0 days 01:38:47.957000
SpeedI1                                 255.0
SpeedI2                                 262.0
SpeedFL                                 280.0
SpeedST                                 246.0
IsPersonalBest                          False
Compound                                 HARD
TyreLife                   

/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/3175505343.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_df["prev_compound"]=driver_df["Compound"].shift(1)
/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/3175505343.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_df["prev_compound"]=driver_df["Compound"].shift(1)
/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/3175505343.py:27: SettingWithCopyWarning: 
A value is trying to be set on

True
Hello1
125358
True
True
Hello1
125359
True
True
Hello1
125360
True
True
Hello1
125361
True
True
Hello1
125362
True
True
Hello1
125363
True
True
Hello1
125364
True
True
Hello1
125365
True
True
Hello1
125366
True
True
Hello2
Unnamed: 0                             125377
Time                   0 days 01:17:44.994000
Driver                                    TSU
DriverNumber                               22
LapTime                0 days 00:01:40.506000
LapNumber                                11.0
Stint                                     2.0
PitOutTime             0 days 01:16:23.901000
PitInTime                                 NaN
Sector1Time            0 days 00:00:43.412000
Sector2Time            0 days 00:00:33.027000
Sector3Time            0 days 00:00:24.067000
Sector1SessionTime     0 days 01:16:47.864000
Sector2SessionTime     0 days 01:17:20.891000
Sector3SessionTime     0 days 01:17:44.958000
SpeedI1                                 254.0
SpeedI2                             

,Unnamed: 0,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Location,Year,second_compound
124447,124447,0 days 01:03:40.152000,VER,1,NaN,1.0,1.0,0 days 00:13:44.843000,NaN,NaN,0 days 00:00:32.084000,0 days 00:00:23.926000,NaN,0 days 01:03:16.243000,0 days 01:03:40.243000,256.0,261.0,276.0,275.0,False,MEDIUM,1.0,True,Red Bull Racing,0 days 01:02:15.963000,2023-06-04 13:03:15.970,1.0,1.0,False,NaN,False,False,Barcelona,2023,False
124448,124448,0 days 01:05:00.554000,VER,1,0 days 00:01:20.402000,2.0,1.0,NaN,NaN,0 days 00:00:24.186000,0 days 00:00:32.088000,0 days 00:00:24.128000,0 days 01:04:04.338000,0 days 01:04:36.426000,0 days 01:05:00.554000,252.0,257.0,276.0,295.0,True,MEDIUM,2.0,True,Red Bull Racing,0 days 01:03:40.152000,2023-06-04 13:04:40.159,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124449,124449,0 days 01:06:21.053000,VER,1,0 days 00:01:20.499000,3.0,1.0,NaN,NaN,0 days 00:00:24.167000,0 days 00:00:32.191000,0 days 00:00:24.141000,0 days 01:05:24.721000,0 days 01:05:56.912000,0 days 01:06:21.053000,249.0,256.0,276.0,297.0,False,MEDIUM,3.0,True,Red Bull Racing,0 days 01:05:00.554000,2023-06-04 13:06:00.561,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124450,124450,0 days 01:07:41.399000,VER,1,0 days 00:01:20.346000,4.0,1.0,NaN,NaN,0 days 00:00:24.022000,0 days 00:00:32.159000,0 days 00:00:24.165000,0 days 01:06:45.075000,0 days 01:07:17.234000,0 days 01:07:41.399000,255.0,256.0,276.0,300.0,True,MEDIUM,4.0,True,Red Bull Racing,0 days 01:06:21.053000,2023-06-04 13:07:21.060,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124451,124451,0 days 01:09:01.682000,VER,1,0 days 00:01:20.283000,5.0,1.0,NaN,NaN,0 days 00:00:24.034000,0 days 00:00:32.213000,0 days 00:00:24.036000,0 days 01:08:05.433000,0 days 01:08:37.646000,0 days 01:09:01.682000,254.0,256.0,277.0,301.0,True,MEDIUM,5.0,True,Red Bull Racing,0 days 01:07:41.399000,2023-06-04 13:08:41.406,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124452,124452,0 days 01:10:22.084000,VER,1,0 days 00:01:20.402000,6.0,1.0,NaN,NaN,0 days 00:00:24.009000,0 days 00:00:32.178000,0 days 00:00:24.215000,0 days 01:09:25.691000,0 days 01:09:57.869000,0 days 01:10:22.084000,255.0,253.0,276.0,NaN,False,MEDIUM,6.0,True,Red Bull Racing,0 days 01:09:01.682000,2023-06-04 13:10:01.689,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124453,124453,0 days 01:11:42.560000,VER,1,0 days 00:01:20.476000,7.0,1.0,NaN,NaN,0 days 00:00:24.092000,0 days 00:00:32.224000,0 days 00:00:24.160000,0 days 01:10:46.176000,0 days 01:11:18.400000,0 days 01:11:42.560000,258.0,259.0,278.0,NaN,False,MEDIUM,7.0,True,Red Bull Racing,0 days 01:10:22.084000,2023-06-04 13:11:22.091,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124454,124454,0 days 01:13:02.454000,VER,1,0 days 00:01:19.894000,8.0,1.0,NaN,NaN,0 days 00:00:23.827000,0 days 00:00:31.980000,0 days 00:00:24.087000,0 days 01:12:06.387000,0 days 01:12:38.367000,0 days 01:13:02.454000,256.0,264.0,276.0,NaN,True,MEDIUM,8.0,True,Red Bull Racing,0 days 01:11:42.560000,2023-06-04 13:12:42.567,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124455,124455,0 days 01:14:22.736000,VER,1,0 days 00:01:20.282000,9.0,1.0,NaN,NaN,0 days 00:00:23.888000,0 days 00:00:32.180000,0 days 00:00:24.214000,0 days 01:13:26.342000,0 days 01:13:58.522000,0 days 01:14:22.736000,259.0,264.0,276.0,NaN,False,MEDIUM,9.0,True,Red Bull Racing,0 days 01:13:02.454000,2023-06-04 13:14:02.461,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124456,124456,0 days 01:15:43.070000,VER,1,0 days 00:01:20.334000,10.0,1.0,NaN,NaN,0 days 00:00:23.937000,0 days 00:00:32.197000,0 days 00:00:24.200000,0 days 01:14:46.673000,0 days 01:15:18.870000,0 days 01:15:43.070000,261.0,265.0,277.0,302.0,False,MEDIUM

In [52]:
start_before = big_laps[(big_laps['LapNumber']==4)]
start_before.columns

Index(['Unnamed: 0', 'Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber',
       'Stint', 'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time',
       'Sector3Time', 'Sector1SessionTime', 'Sector2SessionTime',
       'Sector3SessionTime', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST',
       'IsPersonalBest', 'Compound', 'TyreLife', 'FreshTyre', 'Team',
       'LapStartTime', 'LapStartDate', 'TrackStatus', 'Position', 'Deleted',
       'DeletedReason', 'FastF1Generated', 'IsAccurate', 'Location', 'Year'],
      dtype='object')

In [40]:
import math
import numpy as np

#def track_angle(prev_x, prev_y, curr_x, curr_y, next_x, next_y):
def track_angle(row):
    print(row)
    a = np.array([row['X_prev'],row['Y_prev']])
    b = np.array([row['X'],row['Y']])
    c = np.array([row['X_next'],row['Y_next']])

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)

In [13]:
gasly = session.get_driver("OCO")
#weather_data = session.weather_data
#control_msg = session.race_control_messages

def add_standardized_time(df):
    df['Time_min']=df['Time'].astype('timedelta64[m]')

miami_laps = session.laps
tele = session.get_driver("OCO")
gasly = miami_laps.pick_driver('OCO')
gasly_car = gasly.get_car_data()
gasly_pos = gasly.get_pos_data()
prev = gasly_pos.shift(1)
next = gasly_pos.shift(-1)
gasly_pos = gasly_pos.merge(prev,on="Date",how="left",suffixes=(None,"_prev"))
gasly_pos = gasly_pos.merge(next,on="Date",how="left",suffixes=(None,"_next"))
gasly_data = gasly_car.copy()
gasly_pos.info()
add_standardized_time(gasly_data)
add_standardized_time(weather_data)
#gasly_data['Time_min']=gasly_data['Time'].astype('timedelta64[m]')
#weather_data['Time_min']=weather_data['Time'].astype('timedelta64[m]')
gasly_data = gasly_data.merge(weather_data,on="Time_min",how="left")
#gasly_data = gasly_data.merge(gasly_pos, on="Time",how="left")
control_msg['Date']=control_msg['Time']
#gasly_data = gasly_data.merge(control_msg,on="Date",how="left")
gasly_data.to_csv("test.csv")
#gasly_pos['angle'] = gasly_pos.apply(lambda x: track_angle(x))

gasly_pos


DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`

In [7]:
fig = px.scatter(x=gasly_pos['X'],y=gasly_pos['Y'])#,animation_frame=gasly_pos.index,animation_group=gasly_pos['Source'])
fig.show()